In [13]:
import requests
import json
from csv import writer
import pandas as pd

with open("getEarlyWalletsCOUNT.txt", "r") as f:
    rowNum = int(f.read())

count = rowNum + 1

#GLOBAL VARIABLES
uniswapV2 = "0x7a250d5630b4cf539739df2c5dacb4c659f2488d"
uniswapV3 = "0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45"

# change file here
addressLists = pd.read_csv("tokens.csv")
walletsAndVersions = {}

for i, row in addressLists[rowNum:].iterrows():
    # while count < 2:
    token = row["TokenPair"]

    variables = {"input": token}
    query = """
            query($input: String!){
                swaps(
                    where: {pair_: {id: $input}}
                    orderDirection: asc
                    orderBy: timestamp
                    first: 1000
                ) {
                    sender
                    to
                }
            }
        """

    resp = requests.post('https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2',
                           json={'query': query, 'variables': variables})

    json_data = json.loads(resp.text)
    
    json_data = json_data["data"]["swaps"]

    for eachData in json_data:
        if(eachData["sender"] == uniswapV2 and eachData["to"] != uniswapV2):
            walletsAndVersions[eachData["to"]] = 2
        
        elif (eachData["sender"] == uniswapV3 and eachData["to"] != uniswapV3):
            walletsAndVersions[eachData["to"]] = 3

    count += 1

    with open("getEarlyWalletsCOUNT.txt", "w") as f:
        f.write(str(count-1))

with open('wallets.csv', 'a') as fd:
    for key in walletsAndVersions:
        fd.write(key + "," + str(walletsAndVersions[key]) + "\n")
